In [3]:
"""
Transfer learning by training the network on all the train data (i.e. all movements during the 24hr period, rather than
only the final movements prior to the target time), and then fine-tuning with the final location data only. This script
implements the initial training and saving of a DNN which is later fine tuned ('transfer_learning.ipynb')
"""
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
def accuracy_f_score(y_pred,y_true):
    print(f"Accuracy score: {round(accuracy_score(y_true, y_pred) * 100,2)}%")
    print('\033[92m' + f"F1 score: {f1_score(y_true, y_pred)}" + '\033[0m')
    
def in_city(x_pred,y_pred):
    targets = []
    
    if (3750901.5068 <= x_pred <= 3770901.5069) and (-19268905.6133 <= y_pred <= -19208905.6133):
        return 1
    else:
        return 0

def sigmoid(x):
    e = np.exp(1)
    y = 1/(1+e**(-x))
    return y

def journey_time(x,y):
    """
    Compute journey time in seconds.
    """
    x = pd.to_datetime(x)
    y = pd.to_datetime(y)
    return (y-x).total_seconds()

def to_binary(x):
    result = []
    for n in x:
        result.append(np.argmax(n))
    return result

In [5]:
df = pd.read_csv('../data_train/all_features.csv')
df.head()

,trajectory_id,time_entry,time_exit,x_entry,y_entry,dist,net_tr,prev_tr,x_home,y_home,nj,dist_pct_ch,j_time
0,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,3.773118e+06,-1.914490e+07,94796.788991,46383.455713,31284.419082,3.773413e+06,-1.909828e+07,5.0,-0.248129,420.0
1,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,3.769978e+06,-1.934136e+07,102860.555932,-8447.209016,-8447.209016,3.771380e+06,-1.933274e+07,3.0,0.089470,107.0
2,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,15:26:08,3.757468e+06,-1.923860e+07,3447.328292,135037.335584,74302.066130,3.760880e+06,-1.910042e+07,3.0,-0.955661,0.0
3,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:35:18,15:42:05,3.760505e+06,-1.935500e+07,116098.353965,-38952.875270,-39435.784895,3.751328e+06,-1.916236e+07,3.0,0.514407,407.0
4,traj_00063a4f6c12e1e4de7d876580620667_3_4,14:54:07,15:05:14,3.766319e+06,-1.917013e+07,68990.640078,-27180.691631,-10903.571713,3.747364e+06,-1.927846e+07,4.0,0.187711,667.0


In [11]:
df = pd.read_csv('../data_train/all_binary_features.csv')
df.set_index("trajectory_id", inplace=True)
y = df["final_loc"].values
df.drop(["dist","dist_pct_ch","j_time","final_loc"], axis=1, inplace=True)
df.head()

,dpc,home,start_CC,net_tr_b,prev_tr_b,odd_even_nj,dist_scaled,jt_scaled
trajectory_id,,,,,,,,
traj_0000a8602cf2def930488dee7cdad104_1_0,0.500000,0,0,1,0,1,0.739870,0.009806
traj_0000a8602cf2def930488dee7cdad104_1_1,0.398189,0,0,1,1,1,0.434215,0.012532
traj_0000a8602cf2def930488dee7cdad104_1_2,0.417767,0,0,1,1,1,0.290016,0.024128
traj_0000a8602cf2def930488dee7cdad104_1_3,0.494620,0,0,1,1,1,0.283770,0.047728
traj_0000a8602cf2def930488dee7cdad104_1_4,0.471354,0,0,1,1,1,0.251197,0.000000


In [20]:
df = df.apply(abs,axis=1)
df = df.apply(np.log10,axis=1)
df[df == -np.inf] = 0.0
df.fillna(0, inplace=True)
df.head()

,x_entry,y_entry,dist,net_tr,prev_tr,x_home,y_home,nj,j_time,dpc
trajectory_id,,,,,,,,,,
traj_0000a8602cf2def930488dee7cdad104_1_0,6.574149,7.280896,5.162152,4.997665,3.54961,6.574149,7.280896,0.778151,2.382017,0.000000
traj_0000a8602cf2def930488dee7cdad104_1_1,6.573329,7.286063,4.930779,4.997665,3.54961,6.574149,7.280896,0.778151,2.488551,-0.399910
traj_0000a8602cf2def930488dee7cdad104_1_2,6.573436,7.285412,4.755589,4.997665,3.54961,6.574149,7.280896,0.778151,2.773055,-0.379066
traj_0000a8602cf2def930488dee7cdad104_1_3,6.573438,7.285384,4.746139,4.997665,3.54961,6.574149,7.280896,0.778151,3.069298,-0.305729
traj_0000a8602cf2def930488dee7cdad104_1_4,6.573441,7.285233,4.693225,4.997665,3.54961,6.574149,7.280896,0.778151,0.000000,-0.326653


In [15]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler(feature_range=(-1,1))
X = df.values

In [11]:
"""
Random over-sampling for pre-training the network
"""
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [12]:
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 571050), (1, 571050)]


In [14]:
from keras.callbacks import LearningRateScheduler
import math
def step_decay(epoch):
    initial_lrate = .005
    drop = 0.25
    epochs_drop = 2
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))

loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)

In [16]:
NN_model = Sequential()


# The Input Layer :
NN_model.add(Dense(256, kernel_initializer='normal',input_dim = X.shape[1], activation=None, name='layer1'))
NN_model.add(keras.layers.LeakyReLU(alpha=0.3, name='activation_1'))
NN_model.add(BatchNormalization(name='BN1'))
NN_model.add(Dropout(0.1, name='DO1'))

# The Hidden Layers :
NN_model.add(Dense(1024, kernel_initializer='normal',activation=None, kernel_regularizer=keras.regularizers.l1(0.000),
                   name='fc1'))
NN_model.add(keras.layers.LeakyReLU(alpha=0.3, name='activation_2'))
NN_model.add(BatchNormalization(name='BN2'))
NN_model.add(Dropout(0.1, name='DO2'))


NN_model.add(Dense(1024, kernel_initializer='normal',activation=None, kernel_regularizer=keras.regularizers.l1(0.000),
                   name='fc2'))
NN_model.add(keras.layers.LeakyReLU(alpha=0.3, name='activation_3'))
NN_model.add(BatchNormalization(name='BN3'))
NN_model.add(Dropout(0.1, name='DO3'))

NN_model.add(Dense(1024, kernel_initializer='normal',activation=None, kernel_regularizer=keras.regularizers.l1(0.000),
                   name='fc3'))
NN_model.add(keras.layers.LeakyReLU(alpha=0.3, name='activation_4'))
NN_model.add(BatchNormalization(name='BN4'))


# The Output Layer :
NN_model.add(Dense(2, activation='softmax', name='classifier'))

adam = keras.optimizers.Adam(lr=.0, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# Compile the network :
NN_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
NN_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 256)               2304      
_________________________________________________________________
activation_1 (LeakyReLU)     (None, 256)               0         
_________________________________________________________________
BN1 (BatchNormalization)     (None, 256)               1024      
_________________________________________________________________
DO1 (Dropout)                (None, 256)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              263168    
_________________________________________________________________
activation_2 (LeakyRe

In [20]:
y_binary = to_categorical(y)

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint,loss_history,lrate]
class_weight = {0: .5,
                1: .66}
x_train,x_test,y_train,y_test = train_test_split(X,y_binary,test_size=0,random_state=410,shuffle=True)

NN_model.fit(x_train, y_train, epochs=50, batch_size=256, validation_split = 0.2,
             callbacks=callbacks_list, class_weight=class_weight) # -- not used because of oversampling

Instructions for updating:
Use tf.cast instead.
Train on 651409 samples, validate on 162853 samples
Epoch 1/50
651409/651409 [==============================] - 301s 462us/step - loss: 0.1079 - acc: 0.9324 - val_loss: 0.0962 - val_acc: 0.9372

Epoch 00001: val_loss improved from inf to 0.09620, saving model to Weights-001--0.09620.hdf5
Epoch 2/50
651409/651409 [==============================] - 296s 454us/step - loss: 0.0923 - acc: 0.9350 - val_loss: 0.0908 - val_acc: 0.9377

Epoch 00002: val_loss improved from 0.09620 to 0.09084, saving model to Weights-002--0.09084.hdf5
Epoch 3/50
651409/651409 [==============================] - 290s 445us/step - loss: 0.0923 - acc: 0.9347 - val_loss: 0.0890 - val_acc: 0.9369

Epoch 00003: val_loss improved from 0.09084 to 0.08904, saving model to Weights-003--0.08904.hdf5
Epoch 4/50
651409/651409 [==============================] - 287s 440us/step - loss: 0.0906 - acc: 0.9356 - val_loss: 0.0879 - val_acc: 0.9379

Epoch 00004: val_loss improved from 0.

In [21]:
"""
Select the opimal weights and save the model for further transfer learning
"""

weights_file = 'Weights-036--0.08642.hdf5' # choose the best checkpoint
NN_model.load_weights(weights_file) # load it
NN_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

NN_model.save('pretrained_model_wed.h5')